In [ ]:
%%cmd
python -m pip install pandas

In [ ]:
import pandas as pd

In [ ]:
files = [
    '../../a_ray_tracing_inline_perframe.csv',
    '../../a_ray_tracing_pipeline_perframe.csv',
    '../../b_ray_tracing_inline_perframe.csv',
    '../../b_ray_tracing_pipeline_perframe.csv',
]

In [ ]:
scores = {}

for file in files:
    # Read csv
    file_scores = pd.read_csv(file)
    # Remove empty results
    file_scores = file_scores.loc[:, (file_scores != 0).any(axis=0)]
    scores[file] = file_scores
# Reshape into row per input file + sequence time, column per metric
scores = pd.concat(scores).reset_index().set_index(['level_0', 'Sequence Time (ns)']).drop(['level_1', 'Loop Index', 'Frame'], axis=1)
scores

In [ ]:
# Reshape into row per sequence time + metric, column per input file
new_scores = scores.stack().unstack(0).reset_index()
# ns to s for the sequence time
new_scores['Sequence Time (s)'] = new_scores['Sequence Time (ns)'] / 1_000_000_000
new_scores = new_scores.drop('Sequence Time (ns)', axis=1)
new_scores

In [ ]:
for graph_name in new_scores['level_1'].unique():
    graph_data = new_scores[new_scores['level_1'] == graph_name]
    graph_data = graph_data.drop('level_1', axis=1)
    graph_data = graph_data.apply(pd.to_numeric, errors='coerce').interpolate(method='linear')
    
    graph_data.plot(
        x='Sequence Time (s)', 
        ylabel='value',
        xlabel='benchmark timeline in seconds', 
        kind="line", 
        logy=True,
        figsize=(20,5), 
        colormap='Dark2', 
        grid=True, 
        rot=0,
        title=graph_name
    )